In [1]:
!pip install psycopg2
import pandas as pd
import numpy as np
import xmltodict
import collections
import xml.etree.ElementTree as ET
import psycopg2 as ps
#from config import config

In [2]:
#load datasets
matchdata = pd.read_csv('challenge3_data/matchdata.csv',encoding='latin-1',skipinitialspace = True)
players_attributes = pd.read_csv('challenge3_data/players_attributes.csv')

In [3]:
matchdata.head()

,id,league_id,league_name,season,date,home_team_id,home_team_short_name,home_team_long_name,away_team_id,away_team_short_name,...,away_player_10,away_player_11,goal,shoton,shotoff,foulcommit,card,cross,corner,possession
0,7049,4769,France Ligue 1,2014/2015,08/08/2014 00:00,9837,REI,Stade de Reims,9847,PSG,...,35724.0,49677.0,<goal><value><comment>n</comment><stats><goals...,<shoton><value><stats><blocked>1</blocked></st...,<shotoff><value><stats><shotoff>1</shotoff></s...,<foulcommit><value><stats><foulscommitted>1</f...,<card><value><comment>y</comment><stats><ycard...,<cross><value><stats><crosses>1</crosses></sta...,<corner><value><stats><corners>1</corners></st...,<possession><value><comment>36</comment><event...
1,7050,4769,France Ligue 1,2014/2015,09/08/2014 00:00,7794,BAS,SC Bastia,8592,MAR,...,254308.0,38098.0,<goal><value><comment>n</comment><stats><goals...,<shoton><value><stats><blocked>1</blocked></st...,<shotoff><value><stats><shotoff>1</shotoff></s...,<foulcommit><value><stats><foulscommitted>1</f...,<card><value><comment>y</comment><stats><ycard...,<cross><value><stats><crosses>1</crosses></sta...,<corner><value><stats><corners>1</corners></st...,<possession><value><comment>40</comment><event...
2,7051,4769,France Ligue 1,2014/2015,09/08/2014 00:00,4087,ETG,Évian Thonon Gaillard FC,7819,CAE,...,289155.0,210466.0,<goal><value><comment>n</comment><stats><goals...,<shoton><value><stats><blocked>1</blocked></st...,<shotoff><value><stats><shotoff>1</shotoff></s...,<foulcommit><value><stats><foulscommitted>1</f...,<card><value><comment>y</comment><stats><ycard...,<cross><value><event_incident_typefk>123</even...,<corner><value><stats><corners>1</corners></st...,<possession><value><comment>63</comment><event...
3,7052,4769,France Ligue 1,2014/2015,09/08/2014 00:00,9747,GUI,En Avant de Guingamp,9853,ETI,...,46349.0,26374.0,<goal><value><comment>p</comment><stats><penal...,<shoton><value><stats><shoton>1</shoton></stat...,<shotoff><value><stats><shotoff>1</shotoff></s...,<foulcommit><value><stats><foulscommitted>1</f...,<card><value><comment>y</comment><stats><ycard...,<cross><value><stats><crosses>1</crosses></sta...,<corner><value><stats><corners>1</corners></st...,<possession><value><comment>37</comment><event...
4,7056,4769,France Ligue 1,2014/2015,09/08/2014 00:00,10249,MON,Montpellier Hérault SC,9827,BOR,...,41238.0,40707.0,<goal><value><comment>n</comment><stats><goals...,<shoton><value><stats><shoton>1</shoton></stat...,<shotoff><value><stats><shotoff>1</shotoff></s...,<foulcommit><value><stats><foulscommitted>1</f...,<card><value><comment>y</comment><stats><ycard...,<cross><value><stats><crosses>1</crosses></sta...,<corner><value><stats><corners>1</corners></st...,<possession><value><comment>54</comment><event...


In [4]:
matchdata.dtypes

id                        int64
league_id                 int64
league_name              object
season                   object
date                     object
home_team_id              int64
home_team_short_name     object
home_team_long_name      object
away_team_id              int64
away_team_short_name     object
away_team_long_name      object
home_team_goal            int64
away_team_goal            int64
home_player_1             int64
home_player_2           float64
home_player_3           float64
home_player_4           float64
home_player_5           float64
home_player_6           float64
home_player_7           float64
home_player_8           float64
home_player_9           float64
home_player_10          float64
home_player_11          float64
away_player_1           float64
away_player_2           float64
away_player_3             int64
away_player_4           float64
away_player_5           float64
away_player_6           float64
away_player_7           float64
away_pla

In [5]:
#Identify missing values
matchdata.isna().sum()

id                       0
league_id                0
league_name              0
season                   0
date                     0
home_team_id             0
home_team_short_name     0
home_team_long_name      0
away_team_id             0
away_team_short_name     0
away_team_long_name      0
home_team_goal           0
away_team_goal           0
home_player_1            0
home_player_2            3
home_player_3            1
home_player_4            2
home_player_5            1
home_player_6            4
home_player_7            3
home_player_8            1
home_player_9            3
home_player_10           8
home_player_11           9
away_player_1            1
away_player_2            3
away_player_3            0
away_player_4            2
away_player_5            2
away_player_6            4
away_player_7            2
away_player_8            4
away_player_9            2
away_player_10           4
away_player_11          17
goal                     0
shoton                   0
s

In [6]:
#get numeric columns
numeric = matchdata.select_dtypes(include=np.number)
numeric_columns = numeric.columns

In [7]:
#fill missing values with 0
matchdata[numeric_columns] = matchdata[numeric_columns].fillna(0)

In [8]:
matchdata.isna().sum()

id                      0
league_id               0
league_name             0
season                  0
date                    0
home_team_id            0
home_team_short_name    0
home_team_long_name     0
away_team_id            0
away_team_short_name    0
away_team_long_name     0
home_team_goal          0
away_team_goal          0
home_player_1           0
home_player_2           0
home_player_3           0
home_player_4           0
home_player_5           0
home_player_6           0
home_player_7           0
home_player_8           0
home_player_9           0
home_player_10          0
home_player_11          0
away_player_1           0
away_player_2           0
away_player_3           0
away_player_4           0
away_player_5           0
away_player_6           0
away_player_7           0
away_player_8           0
away_player_9           0
away_player_10          0
away_player_11          0
goal                    0
shoton                  0
shotoff                 0
foulcommit  

In [9]:
matchdata.goal.unique()[1]

'<goal><value><comment>n</comment><stats><goals>1</goals><shoton>1</shoton></stats><event_incident_typefk>6</event_incident_typefk><coordinates><value>21</value><value>16</value></coordinates><elapsed>9</elapsed><subtype>volley</subtype><player1>192711</player1><sortorder>1</sortorder><team>7794</team><id>3633614</id><n>222</n><type>goal</type><goal_type>n</goal_type></value><value><comment>n</comment><stats><goals>1</goals><shoton>1</shoton></stats><event_incident_typefk>406</event_incident_typefk><coordinates><value>25</value><value>64</value></coordinates><elapsed>12</elapsed><player2>280965</player2><subtype>header</subtype><player1>38098</player1><sortorder>0</sortorder><team>8592</team><id>3633655</id><n>225</n><type>goal</type><goal_type>n</goal_type></value><value><comment>o</comment><stats><owngoals>1</owngoals></stats><event_incident_typefk>115</event_incident_typefk><coordinates><value>19</value><value>63</value></coordinates><elapsed>17</elapsed><player1>31014</player1><sor

In [10]:
def parse_goal(goal, home_id, away_id):
    '''
    The function parses the goal values from xml text to a tuple
    Args:
        goal -> xml text with multiple tags and goal info
        home_id -> the id of the home team of the match that goal was scored in
        away_id -> the id of the away team
    Returns:
        a list of two lists: the first one is the home goals list and the second is the away goals list.
        each list consists of a number of lists correspond to each goal.
        list format: (time of the goal in mins-int, scorer id-int, assisstant id-int, goal type-string)
    '''
    if pd.notna(goal):
        if xmltodict.parse(goal)['goal'] != None:
            goal_dict = xmltodict.parse(goal)['goal']['value']
            #print(goal_dict)
            home_goals = list()
            away_goals = list()
            home_time_elapse = list()
            #away_time_elapse = list()
            if type(goal_dict) == collections.OrderedDict:
                goal_dict = [goal_dict]
            for g in goal_dict:
                try:
                    p1 = g['player1']
                except:
                    p1 = "0"
                try:
                    p2 = g['player2']
                except:
                    p2 = "0"                
                g_info = [g['elapsed'],p1, p2, g['comment']]
                if 'del' not in g.keys():
                    if int(g['team']) == home_id:
                        home_goals.append(g_info)
                        #home_time_elapse.append(g_info)
                    else:
                        away_goals.append(g_info)
                        #away_time_elapse.append(g_info)
            return home_goals, away_goals

In [11]:
matchdata["goals_info"] = matchdata.apply(lambda row: parse_goal(row.goal, row.home_team_id, row.away_team_id), axis= 1)

In [12]:
odd_goals = matchdata[matchdata.goals_info.isnull() == False]

In [13]:
odd_goals[odd_goals.home_team_goal+odd_goals.away_team_goal != odd_goals.goals_info.apply(lambda x: len(x[0])+len(x[1]))][['id','goals_info', 'home_team_goal', 'away_team_goal']].head()

,id,goals_info,home_team_goal,away_team_goal
0,7049,"([[22, 94321, 89476, n], [34, 46653, 159839, n...",2,2
13,4015,"([[84, 187266, 0, npm]], [[52, 163200, 21864, ...",0,1
16,4018,"([[30, 37169, 0, npm]], [[90, 361315, 194165, ...",0,1
21,7168,"([[10, 279879, 21414, n], [45, 210184, 0, n], ...",2,1
25,7163,"([[72, 0, 0, dg]], [[19, 40659, 128456, n], [6...",0,2


In [14]:
matchdata

,id,league_id,league_name,season,date,home_team_id,home_team_short_name,home_team_long_name,away_team_id,away_team_short_name,...,away_player_11,goal,shoton,shotoff,foulcommit,card,cross,corner,possession,goals_info
0,7049,4769,France Ligue 1,2014/2015,08/08/2014 00:00,9837,REI,Stade de Reims,9847,PSG,...,49677.0,<goal><value><comment>n</comment><stats><goals...,<shoton><value><stats><blocked>1</blocked></st...,<shotoff><value><stats><shotoff>1</shotoff></s...,<foulcommit><value><stats><foulscommitted>1</f...,<card><value><comment>y</comment><stats><ycard...,<cross><value><stats><crosses>1</crosses></sta...,<corner><value><stats><corners>1</corners></st...,<possession><value><comment>36</comment><event...,"([[22, 94321, 89476, n], [34, 46653, 159839, n..."
1,7050,4769,France Ligue 1,2014/2015,09/08/2014 00:00,7794,BAS,SC Bastia,8592,MAR,...,38098.0,<goal><value><comment>n</comment><stats><goals...,<shoton><value><stats><blocked>1</blocked></st...,<shotoff><value><stats><shotoff>1</shotoff></s...,<foulcommit><value><stats><foulscommitted>1</f...,<card><value><comment>y</comment><stats><ycard...,<cross><value><stats><crosses>1</crosses></sta...,<corner><value><stats><corners>1</corners></st...,<possession><value><comment>40</comment><event...,"([[9, 192711, 0, n], [17, 31014, 0, o], [66, 2..."
2,7051,4769,France Ligue 1,2014/2015,09/08/2014 00:00,4087,ETG,Évian Thonon Gaillard FC,7819,CAE,...,210466.0,<goal><value><comment>n</comment><stats><goals...,<shoton><value><stats><blocked>1</blocked></st...,<shotoff><value><stats><shotoff>1</shotoff></s...,<foulcommit><value><stats><foulscommitted>1</f...,<card><value><comment>y</comment><stats><ycard...,<cross><value><event_incident_typefk>123</even...,<corner><value><stats><corners>1</corners></st...,<possession><value><comment>63</comment><event...,"([], [[12, 319300, 289155, n], [32, 210466, 0,..."
3,7052,4769,France Ligue 1,2014/2015,09/08/2014 00:00,9747,GUI,En Avant de Guingamp,9853,ETI,...,26374.0,<goal><value><comment>p</comment><stats><penal...,<shoton><value><stats><shoton>1</shoton></stat...,<shotoff><value><stats><shotoff>1</shotoff></s...,<foulcommit><value><stats><foulscommitted>1</f...,<card><value><comment>y</comment><stats><ycard...,<cross><value><stats><crosses>1</crosses></sta...,<corner><value><stats><corners>1</corners></st...,<possession><value><comment>37</comment><event...,"([], [[39, 26374, 0, p], [90, 26374, 0, n]])"
4,7056,4769,France Ligue 1,2014/2015,09/08/2014 00:00,10249,MON,Montpellier Hérault SC,9827,BOR,...,40707.0,<goal><value><comment>n</comment><stats><goals...,<shoton><value><stats><shoton>1</shoton></stat...,<shotoff><value><stats><shotoff>1</shotoff></s...,<foulcommit><value><stats><foulscommitted>1</f...,<card><value><comment>y</comment><stats><ycard...,<cross><value><stats><crosses>1</crosses></sta...,<corner><value><stats><corners>1</corners></st...,<possession><value><comment>54</comment><event...,"([], [[17, 40707, 352833, n]])"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1885,12827,10257,Italy Serie A,2014/2015,31/05/2015 00:00,8636,INT,Inter,8534,EMP,...,282723.0,<goal><value><comment>n</comment><stats><goals...,<shoton><value><stats><shoton>1</shoton></stat...,<shotoff><value><stats><shotoff>1</shotoff></s...,<foulcommit><value><stats><foulscommitted>1</f...,<card><value><comment>y</comment><stats><ycard...,<cross><value><stats><crosses>1</crosses></sta...,<corner><value><stats><corners>1</corners></st...,<possession><value><comment>54</comment><stats...,"([[49, 30991, 282770, n], [53, 282770, 239219,..."
1886,12828,10257,Italy Serie A,2014/2015,31/05/2015 00:00,7943,SAS,Sassuolo,10233,GEN,...,166575.0,<goal><value><comment>n</comment><stats><goals...,<shoton><value><stats><shoton>1</shoton></stat...,<shotoff><value><stats><shotoff>1</shotoff></s...,<foulcommit><value><stats><foulscommitted>1</f...,<card><value><comment>y</comment><stats><ycard...,<cross><value><stats><crosses>1</crosses></sta..

In [15]:
matchdata.card.unique()[1]

'<card><value><comment>y</comment><stats><ycards>1</ycards></stats><event_incident_typefk>70</event_incident_typefk><elapsed>19</elapsed><card_type>y</card_type><player1>42693</player1><sortorder>1</sortorder><team>7794</team><n>233</n><type>card</type><id>3633757</id></value><value><comment>y</comment><stats><ycards>1</ycards></stats><event_incident_typefk>70</event_incident_typefk><elapsed>64</elapsed><card_type>y</card_type><player1>280965</player1><sortorder>2</sortorder><team>8592</team><n>288</n><type>card</type><id>3634568</id></value><value><comment>y</comment><stats><ycards>1</ycards></stats><event_incident_typefk>70</event_incident_typefk><elapsed>82</elapsed><card_type>y</card_type><player1>40197</player1><sortorder>1</sortorder><team>7794</team><n>313</n><type>card</type><id>3634847</id></value><value><comment>y</comment><stats><ycards>1</ycards></stats><event_incident_typefk>70</event_incident_typefk><elapsed>87</elapsed><card_type>y</card_type><player1>131991</player1><so

In [16]:
def parse_card(crd):
    '''
    The function parses the card values from xml text to a tuple
    Args:
        crd -> xml text with multiple tags and card info
    Returns:
        a list of lists: each one represents the essential card info
        list format: (time at which the card is recorded in mins-int, card type)
    '''
    if pd.notna(crd):
        if xmltodict.parse(crd)['card'] != None:
            card_dict = xmltodict.parse(crd)['card']['value']
            if type(card_dict) == collections.OrderedDict:
                card_dict = [card_dict]
            card_info = list()
            for c in card_dict:
                try:
                    p1 = c['player1']
                except:
                    p1 = "0"
                c_info = [c['elapsed'], p1, c['comment']] 
                try:
                    card_info.append(c_info)
                except:
                    pass
            return card_info

In [17]:
matchdata['card_info'] = matchdata.card.apply(lambda val: parse_card(val))

In [18]:
matchdata

,id,league_id,league_name,season,date,home_team_id,home_team_short_name,home_team_long_name,away_team_id,away_team_short_name,...,goal,shoton,shotoff,foulcommit,card,cross,corner,possession,goals_info,card_info
0,7049,4769,France Ligue 1,2014/2015,08/08/2014 00:00,9837,REI,Stade de Reims,9847,PSG,...,<goal><value><comment>n</comment><stats><goals...,<shoton><value><stats><blocked>1</blocked></st...,<shotoff><value><stats><shotoff>1</shotoff></s...,<foulcommit><value><stats><foulscommitted>1</f...,<card><value><comment>y</comment><stats><ycard...,<cross><value><stats><crosses>1</crosses></sta...,<corner><value><stats><corners>1</corners></st...,<possession><value><comment>36</comment><event...,"([[22, 94321, 89476, n], [34, 46653, 159839, n...","[[73, 190972, y], [88, 242958, y]]"
1,7050,4769,France Ligue 1,2014/2015,09/08/2014 00:00,7794,BAS,SC Bastia,8592,MAR,...,<goal><value><comment>n</comment><stats><goals...,<shoton><value><stats><blocked>1</blocked></st...,<shotoff><value><stats><shotoff>1</shotoff></s...,<foulcommit><value><stats><foulscommitted>1</f...,<card><value><comment>y</comment><stats><ycard...,<cross><value><stats><crosses>1</crosses></sta...,<corner><value><stats><corners>1</corners></st...,<possession><value><comment>40</comment><event...,"([[9, 192711, 0, n], [17, 31014, 0, o], [66, 2...","[[19, 42693, y], [64, 280965, y], [82, 40197, ..."
2,7051,4769,France Ligue 1,2014/2015,09/08/2014 00:00,4087,ETG,Évian Thonon Gaillard FC,7819,CAE,...,<goal><value><comment>n</comment><stats><goals...,<shoton><value><stats><blocked>1</blocked></st...,<shotoff><value><stats><shotoff>1</shotoff></s...,<foulcommit><value><stats><foulscommitted>1</f...,<card><value><comment>y</comment><stats><ycard...,<cross><value><event_incident_typefk>123</even...,<corner><value><stats><corners>1</corners></st...,<possession><value><comment>63</comment><event...,"([], [[12, 319300, 289155, n], [32, 210466, 0,...","[[48, 35121, y], [90, 128666, y]]"
3,7052,4769,France Ligue 1,2014/2015,09/08/2014 00:00,9747,GUI,En Avant de Guingamp,9853,ETI,...,<goal><value><comment>p</comment><stats><penal...,<shoton><value><stats><shoton>1</shoton></stat...,<shotoff><value><stats><shotoff>1</shotoff></s...,<foulcommit><value><stats><foulscommitted>1</f...,<card><value><comment>y</comment><stats><ycard...,<cross><value><stats><crosses>1</crosses></sta...,<corner><value><stats><corners>1</corners></st...,<possession><value><comment>37</comment><event...,"([], [[39, 26374, 0, p], [90, 26374, 0, n]])","[[38, 277412, y], [55, 94323, y], [73, 26051, ..."
4,7056,4769,France Ligue 1,2014/2015,09/08/2014 00:00,10249,MON,Montpellier Hérault SC,9827,BOR,...,<goal><value><comment>n</comment><stats><goals...,<shoton><value><stats><shoton>1</shoton></stat...,<shotoff><value><stats><shotoff>1</shotoff></s...,<foulcommit><value><stats><foulscommitted>1</f...,<card><value><comment>y</comment><stats><ycard...,<cross><value><stats><crosses>1</crosses></sta...,<corner><value><stats><corners>1</corners></st...,<possession><value><comment>54</comment><event...,"([], [[17, 40707, 352833, n]])","[[7, 139635, y], [33, 154230, y], [63, 355901,..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1885,12827,10257,Italy Serie A,2014/2015,31/05/2015 00:00,8636,INT,Inter,8534,EMP,...,<goal><value><comment>n</comment><stats><goals...,<shoton><value><stats><shoton>1</shoton></stat...,<shotoff><value><stats><shotoff>1</shotoff></s...,<foulcommit><value><stats><foulscommitted>1</f...,<card><value><comment>y</comment><stats><ycard...,<cross><value><stats><crosses>1</crosses></sta...,<corner><value><stats><corners>1</corners></st...,<possession><value><comment>54</comment><stats...,"([[49, 30991, 282770, n], [53, 282770, 239219,...","[[63, 365624, y], [67, 41688, y], [84, 21446, ..."
1886,12828,10257,Italy Serie A,2014/2015,31/05/2015 00:00,7943,SAS,Sassuolo,10233,GEN,...,<goal><value><comment>n</comment><stats><goals...,<shoton><value><stats><shoton>1

In [19]:
matchdata.corner.unique()[1]

'<corner><value><stats><corners>1</corners></stats><event_incident_typefk>329</event_incident_typefk><coordinates><value>2</value><value>69</value></coordinates><elapsed>20</elapsed><subtype>cross</subtype><player1>254308</player1><sortorder>1</sortorder><team>8592</team><n>236</n><type>corner</type><id>3633773</id></value><value><stats><corners>1</corners></stats><event_incident_typefk>329</event_incident_typefk><coordinates><value>2</value><value>69</value></coordinates><elapsed>20</elapsed><subtype>cross</subtype><player1>254308</player1><sortorder>3</sortorder><team>8592</team><n>237</n><type>corner</type><id>3633779</id></value><value><stats><corners>1</corners></stats><event_incident_typefk>329</event_incident_typefk><coordinates><value>44</value><value>1</value></coordinates><elapsed>36</elapsed><subtype>cross</subtype><player1>155623</player1><sortorder>1</sortorder><team>7794</team><n>254</n><type>corner</type><id>3634027</id></value><value><stats><corners>1</corners></stats><

In [20]:
def parse_corner(crner):
    '''
    The function parses the corner values from xml text to a list of tuples
    Args:
        crner -> xml text with multiple tags and corner info
    Returns:
        a list of lists: each one represents the essential corner info
        list format: (time at which the card was recorded in mins-int, player_id-int, id-int)
    '''
    if pd.notna(crner):
        if xmltodict.parse(crner)['corner'] != None:
            corner_dict = xmltodict.parse(crner)['corner']['value']
            if type(corner_dict) == collections.OrderedDict:
                corner_dict = [corner_dict]
            corner_info = list()
            for c in corner_dict:
                try:
                    p1 = int(c['player1'])
                except:
                    p1 = 0
                c_info = [int(c['elapsed']), p1,int(c['id'])]
                try:
                    corner_info.append(c_info)
                except:
                    pass
            return corner_info

In [21]:
matchdata['corner_info'] = matchdata.corner.apply(lambda val: parse_corner(val))

In [22]:
matchdata

,id,league_id,league_name,season,date,home_team_id,home_team_short_name,home_team_long_name,away_team_id,away_team_short_name,...,shoton,shotoff,foulcommit,card,cross,corner,possession,goals_info,card_info,corner_info
0,7049,4769,France Ligue 1,2014/2015,08/08/2014 00:00,9837,REI,Stade de Reims,9847,PSG,...,<shoton><value><stats><blocked>1</blocked></st...,<shotoff><value><stats><shotoff>1</shotoff></s...,<foulcommit><value><stats><foulscommitted>1</f...,<card><value><comment>y</comment><stats><ycard...,<cross><value><stats><crosses>1</crosses></sta...,<corner><value><stats><corners>1</corners></st...,<possession><value><comment>36</comment><event...,"([[22, 94321, 89476, n], [34, 46653, 159839, n...","[[73, 190972, y], [88, 242958, y]]","[[15, 26852, 3631175], [44, 26852, 3631254], [..."
1,7050,4769,France Ligue 1,2014/2015,09/08/2014 00:00,7794,BAS,SC Bastia,8592,MAR,...,<shoton><value><stats><blocked>1</blocked></st...,<shotoff><value><stats><shotoff>1</shotoff></s...,<foulcommit><value><stats><foulscommitted>1</f...,<card><value><comment>y</comment><stats><ycard...,<cross><value><stats><crosses>1</crosses></sta...,<corner><value><stats><corners>1</corners></st...,<possession><value><comment>40</comment><event...,"([[9, 192711, 0, n], [17, 31014, 0, o], [66, 2...","[[19, 42693, y], [64, 280965, y], [82, 40197, ...","[[20, 254308, 3633773], [20, 254308, 3633779],..."
2,7051,4769,France Ligue 1,2014/2015,09/08/2014 00:00,4087,ETG,Évian Thonon Gaillard FC,7819,CAE,...,<shoton><value><stats><blocked>1</blocked></st...,<shotoff><value><stats><shotoff>1</shotoff></s...,<foulcommit><value><stats><foulscommitted>1</f...,<card><value><comment>y</comment><stats><ycard...,<cross><value><event_incident_typefk>123</even...,<corner><value><stats><corners>1</corners></st...,<possession><value><comment>63</comment><event...,"([], [[12, 319300, 289155, n], [32, 210466, 0,...","[[48, 35121, y], [90, 128666, y]]","[[19, 46757, 3633745], [21, 74169, 3633777], [..."
3,7052,4769,France Ligue 1,2014/2015,09/08/2014 00:00,9747,GUI,En Avant de Guingamp,9853,ETI,...,<shoton><value><stats><shoton>1</shoton></stat...,<shotoff><value><stats><shotoff>1</shotoff></s...,<foulcommit><value><stats><foulscommitted>1</f...,<card><value><comment>y</comment><stats><ycard...,<cross><value><stats><crosses>1</crosses></sta...,<corner><value><stats><corners>1</corners></st...,<possession><value><comment>37</comment><event...,"([], [[39, 26374, 0, p], [90, 26374, 0, n]])","[[38, 277412, y], [55, 94323, y], [73, 26051, ...","[[20, 46349, 3633760], [23, 46349, 3633804], [..."
4,7056,4769,France Ligue 1,2014/2015,09/08/2014 00:00,10249,MON,Montpellier Hérault SC,9827,BOR,...,<shoton><value><stats><shoton>1</shoton></stat...,<shotoff><value><stats><shotoff>1</shotoff></s...,<foulcommit><value><stats><foulscommitted>1</f...,<card><value><comment>y</comment><stats><ycard...,<cross><value><stats><crosses>1</crosses></sta...,<corner><value><stats><corners>1</corners></st...,<possession><value><comment>54</comment><event...,"([], [[17, 40707, 352833, n]])","[[7, 139635, y], [33, 154230, y], [63, 355901,...","[[13, 166019, 3633661], [30, 25525, 3633915], ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1885,12827,10257,Italy Serie A,2014/2015,31/05/2015 00:00,8636,INT,Inter,8534,EMP,...,<shoton><value><stats><shoton>1</shoton></stat...,<shotoff><value><stats><shotoff>1</shotoff></s...,<foulcommit><value><stats><foulscommitted>1</f...,<card><value><comment>y</comment><stats><ycard...,<cross><value><stats><crosses>1</crosses></sta...,<corner><value><stats><corners>1</corners></st...,<possession><value><comment>54</comment><stats...,"([[49, 30991, 282770, n], [53, 282770, 239219,...","[[63, 365624, y], [67, 41688, y], [84, 21446, ...","[[6, 21446, 4532573], [22, 21446, 4532792], [3..."
1886,12828,10257,Italy Serie A,2014/2015,31/05/2015 00:00,7943,SAS,Sassuolo,10233,GEN,...,<shoton><value><stats><shoton>1</shoton></stat...,<shotoff><va

In [23]:
#getting data about leafues
league_df = matchdata.iloc[:,1:3].reset_index().drop('index', axis= 1)

In [24]:
league_df = league_df.drop_duplicates()

In [25]:
league_df['league_name'].str.strip()

0               France Ligue 1
10      England Premier League
28       Germany 1. Bundesliga
44             Spain LIGA BBVA
84               Italy Serie A
1332    Netherlands Eredivisie
Name: league_name, dtype: object

In [26]:
league_df

,league_id,league_name
0,4769,France Ligue 1
10,1729,England Premier League
28,7809,Germany 1. Bundesliga
44,21518,Spain LIGA BBVA
84,10257,Italy Serie A
1332,13274,Netherlands Eredivisie


In [27]:
#working with the players' attributes data
players_attributes

,player_id,player_name,birthday,height,weight,date,overall_rating,potential,preferred_foot,attacking_work_rate,...,vision,penalties,marking,standing_tackle,sliding_tackle,gk_diving,gk_handling,gk_kicking,gk_positioning,gk_reflexes
0,155782,Aaron Cresswell,1989-12-15,170.18,146,2015-12-24,73.0,75.0,left,high,...,65.0,59.0,76.0,75.0,78.0,14.0,7.0,9.0,9.0,12.0
1,155782,Aaron Cresswell,1989-12-15,170.18,146,2015-12-17,73.0,75.0,left,high,...,65.0,59.0,76.0,75.0,78.0,14.0,7.0,9.0,9.0,12.0
2,155782,Aaron Cresswell,1989-12-15,170.18,146,2015-10-16,73.0,77.0,left,high,...,65.0,59.0,76.0,75.0,78.0,14.0,7.0,9.0,9.0,12.0
3,155782,Aaron Cresswell,1989-12-15,170.18,146,2015-09-25,74.0,78.0,left,high,...,65.0,59.0,76.0,75.0,78.0,14.0,7.0,9.0,9.0,12.0
4,155782,Aaron Cresswell,1989-12-15,170.18,146,2015-09-21,73.0,77.0,left,medium,...,65.0,59.0,76.0,75.0,78.0,14.0,7.0,9.0,9.0,12.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33572,282473,Zouhair Feddal,1989-01-01,190.50,172,2014-09-18,61.0,65.0,left,medium,...,32.0,33.0,65.0,69.0,53.0,12.0,13.0,11.0,9.0,7.0
33573,282473,Zouhair Feddal,1989-01-01,190.50,172,2014-02-07,62.0,66.0,left,medium,...,33.0,34.0,66.0,70.0,54.0,13.0,14.0,12.0,10.0,8.0
33574,26357,Zoumana Camara,1979-04-03,182.88,168,2015-03-06,73.0,73.0,right,low,...,52.0,50.0,76.0,77.0,72.0,14.0,10.0,11.0,8.0,12.0
33575,26357,Zoumana Camara,1979-04-03,182.88,168,2014-09-18,73.0,73.0,right,low,...,52.0,50.0,76.0,77.0,72.0,14.0,10.0,11.0,8.0,12.0


In [28]:
for col in enumerate(players_attributes.columns):
    print(col)

(0, 'player_id')
(1, 'player_name')
(2, 'birthday')
(3, 'height')
(4, 'weight')
(5, 'date')
(6, 'overall_rating')
(7, 'potential')
(8, 'preferred_foot')
(9, 'attacking_work_rate')
(10, 'defensive_work_rate')
(11, 'crossing')
(12, 'finishing')
(13, 'heading_accuracy')
(14, 'short_passing')
(15, 'volleys')
(16, 'dribbling')
(17, 'curve')
(18, 'free_kick_accuracy')
(19, 'long_passing')
(20, 'ball_control')
(21, 'acceleration')
(22, 'sprint_speed')
(23, 'agility')
(24, 'reactions')
(25, 'balance')
(26, 'shot_power')
(27, 'jumping')
(28, 'stamina')
(29, 'strength')
(30, 'long_shots')
(31, 'aggression')
(32, 'interceptions')
(33, 'positioning')
(34, 'vision')
(35, 'penalties')
(36, 'marking')
(37, 'standing_tackle')
(38, 'sliding_tackle')
(39, 'gk_diving')
(40, 'gk_handling')
(41, 'gk_kicking')
(42, 'gk_positioning')
(43, 'gk_reflexes')


In [29]:
#check for missing values
players_attributes.isna().sum()

player_id               0
player_name             0
birthday                0
height                  0
weight                  0
date                    0
overall_rating         24
potential              24
preferred_foot         24
attacking_work_rate    24
defensive_work_rate    24
crossing               24
finishing              24
heading_accuracy       24
short_passing          24
volleys                24
dribbling              24
curve                  24
free_kick_accuracy     24
long_passing           24
ball_control           24
acceleration           24
sprint_speed           24
agility                24
reactions              24
balance                24
shot_power             24
jumping                24
stamina                24
strength               24
long_shots             24
aggression             24
interceptions          24
positioning            24
vision                 24
penalties              24
marking                24
standing_tackle        24
sliding_tack

In [30]:
#drop all players with no overall rating
players_attributes.dropna(subset=['overall_rating'],axis=0,inplace=True)

In [31]:
print(players_attributes)

       player_id      player_name    birthday  height  weight        date  \
0         155782  Aaron Cresswell  1989-12-15  170.18     146  2015-12-24   
1         155782  Aaron Cresswell  1989-12-15  170.18     146  2015-12-17   
2         155782  Aaron Cresswell  1989-12-15  170.18     146  2015-10-16   
3         155782  Aaron Cresswell  1989-12-15  170.18     146  2015-09-25   
4         155782  Aaron Cresswell  1989-12-15  170.18     146  2015-09-21   
...          ...              ...         ...     ...     ...         ...   
33572     282473   Zouhair Feddal  1989-01-01  190.50     172  2014-09-18   
33573     282473   Zouhair Feddal  1989-01-01  190.50     172  2014-02-07   
33574      26357   Zoumana Camara  1979-04-03  182.88     168  2015-03-06   
33575      26357   Zoumana Camara  1979-04-03  182.88     168  2014-09-18   
33576      26357   Zoumana Camara  1979-04-03  182.88     168  2014-05-16   

       overall_rating  potential preferred_foot attacking_work_rate  ...  \

In [32]:
players_attributes['player_id'].nunique()

3746

In [33]:
#Drop duplicate rows
new_players_attributes = players_attributes.drop_duplicates(subset='player_id', keep="first")

In [34]:
new_players_attributes

,player_id,player_name,birthday,height,weight,date,overall_rating,potential,preferred_foot,attacking_work_rate,...,vision,penalties,marking,standing_tackle,sliding_tackle,gk_diving,gk_handling,gk_kicking,gk_positioning,gk_reflexes
0,155782,Aaron Cresswell,1989-12-15,170.18,146,2015-12-24,73.0,75.0,left,high,...,65.0,59.0,76.0,75.0,78.0,14.0,7.0,9.0,9.0,12.0
12,27316,Aaron Hunt,1986-09-04,182.88,161,2015-09-25,77.0,77.0,left,medium,...,76.0,81.0,28.0,34.0,35.0,15.0,12.0,7.0,16.0,15.0
21,30895,Aaron Lennon,1987-04-16,165.10,139,2015-10-16,77.0,77.0,right,high,...,73.0,62.0,30.0,34.0,35.0,14.0,7.0,7.0,16.0,11.0
31,101042,Aaron Meijers,1987-10-28,175.26,170,2015-12-03,69.0,69.0,left,medium,...,71.0,56.0,67.0,68.0,65.0,7.0,15.0,7.0,10.0,15.0
39,40719,Aaron Niguez,1989-04-26,170.18,143,2015-12-17,71.0,73.0,left,medium,...,62.0,67.0,36.0,31.0,37.0,14.0,13.0,9.0,15.0,10.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33519,35724,Zlatan Ibrahimovic,1981-10-03,195.58,209,2015-09-21,89.0,89.0,right,medium,...,83.0,91.0,15.0,41.0,27.0,13.0,15.0,10.0,9.0,12.0
33525,42860,Zlatko Junuzovic,1987-09-26,172.72,152,2015-11-26,78.0,78.0,right,high,...,78.0,70.0,69.0,70.0,65.0,15.0,9.0,11.0,15.0,8.0
33549,107281,Zoltan Stieber,1988-10-16,175.26,148,2015-11-06,73.0,74.0,left,medium,...,73.0,65.0,26.0,31.0,32.0,6.0,6.0,9.0,16.0,7.0
33560,282473,Zouhair Feddal,1989-01-01,190.50,172,2015-11-06,75.0,77.0,left,medium,...,47.0,48.0,78.0,79.0,68.0,13.0,14.0,12.0,10.0,8.0


In [35]:
#Replace missing values
numeric = players_attributes.select_dtypes(include=np.number)
numeric_columns = numeric.columns

In [36]:
#fill missing values with 0
players_attributes[numeric_columns] = players_attributes[numeric_columns].fillna(0)

In [37]:
players_attributes = players_attributes.fillna("")

In [38]:
players_attributes.isna().sum()

player_id              0
player_name            0
birthday               0
height                 0
weight                 0
date                   0
overall_rating         0
potential              0
preferred_foot         0
attacking_work_rate    0
defensive_work_rate    0
crossing               0
finishing              0
heading_accuracy       0
short_passing          0
volleys                0
dribbling              0
curve                  0
free_kick_accuracy     0
long_passing           0
ball_control           0
acceleration           0
sprint_speed           0
agility                0
reactions              0
balance                0
shot_power             0
jumping                0
stamina                0
strength               0
long_shots             0
aggression             0
interceptions          0
positioning            0
vision                 0
penalties              0
marking                0
standing_tackle        0
sliding_tackle         0
gk_diving              0


In [39]:
#Get player IDs from the matchdata 
player_ids = matchdata[["home_team_id","home_player_1","home_player_2","home_player_3","home_player_4","home_player_5","home_player_6","home_player_7","home_player_8","home_player_9","home_player_10","home_player_11"]]

In [40]:
player_ids.shape

(1890, 12)

In [41]:
player_ids1 = player_ids.drop_duplicates(subset='home_team_id', keep="last")

In [42]:
player_ids1.head()

,home_team_id,home_player_1,home_player_2,home_player_3,home_player_4,home_player_5,home_player_6,home_player_7,home_player_8,home_player_9,home_player_10,home_player_11
1510,9908,118848,121642.0,352392.0,45496.0,442729.0,614453.0,45879.0,352404.0,351898.0,352968.0,570980.0
1511,10235,26002,560852.0,424879.0,299572.0,185554.0,26539.0,97470.0,209384.0,188557.0,38865.0,402975.0
1512,10229,214984,186880.0,245315.0,465628.0,26797.0,157101.0,159465.0,384446.0,209786.0,172388.0,512747.0
1513,8525,156546,26708.0,243481.0,45837.0,352366.0,118843.0,17848.0,145586.0,305831.0,186136.0,143523.0
1514,6433,231674,179242.0,150589.0,21765.0,243429.0,243473.0,110135.0,352023.0,26686.0,17259.0,298449.0


In [43]:
#change datatypes of players from float to int
player_ids1.astype({"home_player_2": int, "home_player_3": int, "home_player_4": int, "home_player_5": int, "home_player_6": int, "home_player_7": int, "home_player_8": int, "home_player_9": int, "home_player_10": int, "home_player_11": int})

,home_team_id,home_player_1,home_player_2,home_player_3,home_player_4,home_player_5,home_player_6,home_player_7,home_player_8,home_player_9,home_player_10,home_player_11
1510,9908,118848,121642,352392,45496,442729,614453,45879,352404,351898,352968,570980
1511,10235,26002,560852,424879,299572,185554,26539,97470,209384,188557,38865,402975
1512,10229,214984,186880,245315,465628,26797,157101,159465,384446,209786,172388,512747
1513,8525,156546,26708,243481,45837,352366,118843,17848,145586,305831,186136,143523
1514,6433,231674,179242,150589,21765,243429,243473,110135,352023,26686,17259,298449
...,...,...,...,...,...,...,...,...,...,...,...,...
1885,8636,42422,98254,25526,27719,197352,239219,49970,206508,21446,30991,282770
1886,7943,74028,178120,32769,73024,41497,39222,74034,27685,364520,166595,195199
1887,8686,178732,264842,18880,103266,24622,237606,30682,239061,155738,30714,79187
1888,9882,41671,25587,18823,362195,163918,213366,163919,317156,181995,33639,196484


In [44]:
#Have a list of player IDs
player_id_list = players_attributes['player_id'].unique().tolist()

In [45]:
cols = np.r_[0, 5:44]
player_attributes_df = new_players_attributes.iloc[:, cols].reset_index().drop('index', axis= 1)

In [46]:
player_attributes_df

,player_id,date,overall_rating,potential,preferred_foot,attacking_work_rate,defensive_work_rate,crossing,finishing,heading_accuracy,...,vision,penalties,marking,standing_tackle,sliding_tackle,gk_diving,gk_handling,gk_kicking,gk_positioning,gk_reflexes
0,155782,2015-12-24,73.0,75.0,left,high,medium,79.0,51.0,57.0,...,65.0,59.0,76.0,75.0,78.0,14.0,7.0,9.0,9.0,12.0
1,27316,2015-09-25,77.0,77.0,left,medium,medium,77.0,72.0,64.0,...,76.0,81.0,28.0,34.0,35.0,15.0,12.0,7.0,16.0,15.0
2,30895,2015-10-16,77.0,77.0,right,high,medium,78.0,66.0,28.0,...,73.0,62.0,30.0,34.0,35.0,14.0,7.0,7.0,16.0,11.0
3,101042,2015-12-03,69.0,69.0,left,medium,medium,63.0,42.0,59.0,...,71.0,56.0,67.0,68.0,65.0,7.0,15.0,7.0,10.0,15.0
4,40719,2015-12-17,71.0,73.0,left,medium,medium,64.0,67.0,54.0,...,62.0,67.0,36.0,31.0,37.0,14.0,13.0,9.0,15.0,10.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3741,35724,2015-09-21,89.0,89.0,right,medium,low,76.0,90.0,76.0,...,83.0,91.0,15.0,41.0,27.0,13.0,15.0,10.0,9.0,12.0
3742,42860,2015-11-26,78.0,78.0,right,high,high,86.0,66.0,58.0,...,78.0,70.0,69.0,70.0,65.0,15.0,9.0,11.0,15.0,8.0
3743,107281,2015-11-06,73.0,74.0,left,medium,medium,69.0,71.0,45.0,...,73.0,65.0,26.0,31.0,32.0,6.0,6.0,9.0,16.0,7.0
3744,282473,2015-11-06,75.0,77.0,left,medium,medium,60.0,40.0,73.0,...,47.0,48.0,78.0,79.0,68.0,13.0,14.0,12.0,10.0,8.0


In [47]:
for col in enumerate(matchdata.columns):
    print(col)

(0, 'id')
(1, 'league_id')
(2, 'league_name')
(3, 'season')
(4, 'date')
(5, 'home_team_id')
(6, 'home_team_short_name')
(7, 'home_team_long_name')
(8, 'away_team_id')
(9, 'away_team_short_name')
(10, 'away_team_long_name')
(11, 'home_team_goal')
(12, 'away_team_goal')
(13, 'home_player_1')
(14, 'home_player_2')
(15, 'home_player_3')
(16, 'home_player_4')
(17, 'home_player_5')
(18, 'home_player_6')
(19, 'home_player_7')
(20, 'home_player_8')
(21, 'home_player_9')
(22, 'home_player_10')
(23, 'home_player_11')
(24, 'away_player_1')
(25, 'away_player_2')
(26, 'away_player_3')
(27, 'away_player_4')
(28, 'away_player_5')
(29, 'away_player_6')
(30, 'away_player_7')
(31, 'away_player_8')
(32, 'away_player_9')
(33, 'away_player_10')
(34, 'away_player_11')
(35, 'goal')
(36, 'shoton')
(37, 'shotoff')
(38, 'foulcommit')
(39, 'card')
(40, 'cross')
(41, 'corner')
(42, 'possession')
(43, 'goals_info')
(44, 'card_info')
(45, 'corner_info')


In [48]:
uniqueValues = matchdata['home_team_id'].unique()

In [49]:
uniqueValues

array([ 9837,  7794,  4087,  9747, 10249,  9830,  9831,  9748,  9829,
        7819,  9825,  8197, 10260, 10172, 10194,  8659,  8654,  8588,
        8550,  9847,  9851,  9941,  8650, 10261,  9827,  8592,  9853,
        8191,  9823,  8455,  9826,  8668, 10003,  8639,  9789,  8226,
        9904,  8177,  9810,  8302,  9864,  7878,  9865,  8667,  8472,
        8586,  9788,  8460,  8581,  9910,  8372,  8634,  8456,  8633,
        8406, 10267,  8305,  8689, 10189,  8178,  8721,  8697, 10269,
        9790,  8533,  8686,  8558,  7869,  9906,  8315, 10252,  8358,
        9880, 10233,  8564,  8540,  7943,  9804,  8600,  8560,  9783,
       10268, 10205,  8466,  8534,  9885,  8529,  8636,  8543,  9875,
       10167,  9882,  8370,  9876,  9905,  8524,  8722,  8535, 10218,
       10217, 10228,  9761,  7788,  6631,  6413,  9791, 10235, 10229,
        6433,  8525,  8593,  8674,  9908,  8277,  8611,  8640],
      dtype=int64)

In [50]:
#get teams data
teams_df = matchdata[["home_team_id","home_team_short_name","home_team_long_name","league_id"]]

In [51]:
teams_df.shape

(1890, 4)

In [52]:
teams_df1 = teams_df.drop_duplicates()

In [53]:
teams_df1.shape

(116, 4)

In [54]:
#final teams data
teams_df1

,home_team_id,home_team_short_name,home_team_long_name,league_id
0,9837,REI,Stade de Reims,4769
1,7794,BAS,SC Bastia,4769
2,4087,ETG,Évian Thonon Gaillard FC,4769
3,9747,GUI,En Avant de Guingamp,4769
4,10249,MON,Montpellier Hérault SC,4769
...,...,...,...,...
1345,8674,GRO,FC Groningen,13274
1348,9908,UTR,FC Utrecht,13274
1352,8277,VIT,Vitesse,13274
1356,8611,TWE,FC Twente,13274


In [55]:
teams_df1 = teams_df1.rename(columns={'home_team_id': 'team_id', 'home_team_short_name': 'team_short_name','home_team_long_name':'team_long_name'})

In [56]:
teams_df1 

,team_id,team_short_name,team_long_name,league_id
0,9837,REI,Stade de Reims,4769
1,7794,BAS,SC Bastia,4769
2,4087,ETG,Évian Thonon Gaillard FC,4769
3,9747,GUI,En Avant de Guingamp,4769
4,10249,MON,Montpellier Hérault SC,4769
...,...,...,...,...
1345,8674,GRO,FC Groningen,13274
1348,9908,UTR,FC Utrecht,13274
1352,8277,VIT,Vitesse,13274
1356,8611,TWE,FC Twente,13274


In [57]:
#Get match stats data
cols1 = np.r_[0:2, 3:6,8,11:13,43:46]
matchstats_df = matchdata.iloc[:, cols1].reset_index().drop('index', axis= 1)

In [58]:
matchstats_df

,id,league_id,season,date,home_team_id,away_team_id,home_team_goal,away_team_goal,goals_info,card_info,corner_info
0,7049,4769,2014/2015,08/08/2014 00:00,9837,9847,2,2,"([[22, 94321, 89476, n], [34, 46653, 159839, n...","[[73, 190972, y], [88, 242958, y]]","[[15, 26852, 3631175], [44, 26852, 3631254], [..."
1,7050,4769,2014/2015,09/08/2014 00:00,7794,8592,3,3,"([[9, 192711, 0, n], [17, 31014, 0, o], [66, 2...","[[19, 42693, y], [64, 280965, y], [82, 40197, ...","[[20, 254308, 3633773], [20, 254308, 3633779],..."
2,7051,4769,2014/2015,09/08/2014 00:00,4087,7819,0,3,"([], [[12, 319300, 289155, n], [32, 210466, 0,...","[[48, 35121, y], [90, 128666, y]]","[[19, 46757, 3633745], [21, 74169, 3633777], [..."
3,7052,4769,2014/2015,09/08/2014 00:00,9747,9853,0,2,"([], [[39, 26374, 0, p], [90, 26374, 0, n]])","[[38, 277412, y], [55, 94323, y], [73, 26051, ...","[[20, 46349, 3633760], [23, 46349, 3633804], [..."
4,7056,4769,2014/2015,09/08/2014 00:00,10249,9827,0,1,"([], [[17, 40707, 352833, n]])","[[7, 139635, y], [33, 154230, y], [63, 355901,...","[[13, 166019, 3633661], [30, 25525, 3633915], ..."
...,...,...,...,...,...,...,...,...,...,...,...
1885,12827,10257,2014/2015,31/05/2015 00:00,8636,8534,4,3,"([[49, 30991, 282770, n], [53, 282770, 239219,...","[[63, 365624, y], [67, 41688, y], [84, 21446, ...","[[6, 21446, 4532573], [22, 21446, 4532792], [3..."
1886,12828,10257,2014/2015,31/05/2015 00:00,7943,10233,3,1,"([[4, 364520, 27685, n], [19, 166595, 364520, ...","[[7, 113318, y], [27, 154239, y], [64, 52243, ...","[[14, 195199, 4532690], [30, 161015, 4532894],..."
1887,12831,10257,2014/2015,31/05/2015 00:00,8686,8540,1,2,"([[85, 30714, 41433, n]], [[35, 270774, 0, p],...","[[56, 103266, y], [68, 108401, y], [77, 41874,...","[[20, 270774, 4532699], [21, 270774, 4532707],..."
1888,12832,10257,2014/2015,31/05/2015 00:00,9882,10167,2,2,"([[54, 362195, 317156, n], [79, 25587, 33639, ...","[[78, 50449, y]]","[[4, 317156, 4532547], [7, 39367, 4532589], [2..."


In [59]:
#Players data
player_df = players_attributes[["player_id","player_name","birthday","height","weight"]]

In [60]:
player_df = player_df.drop_duplicates(subset = "player_id", keep = "first")

In [61]:
player_df

,player_id,player_name,birthday,height,weight
0,155782,Aaron Cresswell,1989-12-15,170.18,146
12,27316,Aaron Hunt,1986-09-04,182.88,161
21,30895,Aaron Lennon,1987-04-16,165.10,139
31,101042,Aaron Meijers,1987-10-28,175.26,170
39,40719,Aaron Niguez,1989-04-26,170.18,143
...,...,...,...,...,...
33519,35724,Zlatan Ibrahimovic,1981-10-03,195.58,209
33525,42860,Zlatko Junuzovic,1987-09-26,172.72,152
33549,107281,Zoltan Stieber,1988-10-16,175.26,148
33560,282473,Zouhair Feddal,1989-01-01,190.50,172


In [62]:
players_attributes['player_id'].nunique()

3746

In [63]:
#Havw a list of player Ids
player_id_list = players_attributes['player_id'].unique().tolist()

In [64]:
player_id_list

[155782,
 27316,
 30895,
 101042,
 40719,
 75489,
 167027,
 128456,
 42664,
 467485,
 563215,
 41093,
 564712,
 67334,
 173955,
 39562,
 191784,
 194479,
 189181,
 352887,
 439366,
 148827,
 196947,
 324910,
 24385,
 29904,
 105548,
 240556,
 114503,
 27277,
 165706,
 213695,
 660145,
 293235,
 495841,
 519835,
 157303,
 38617,
 89240,
 24159,
 37234,
 209372,
 500502,
 185336,
 12245,
 684978,
 160447,
 36774,
 357881,
 181211,
 533212,
 103866,
 47394,
 155738,
 361710,
 45816,
 77741,
 131409,
 132844,
 45815,
 449241,
 261599,
 102572,
 482331,
 40501,
 178574,
 45744,
 23358,
 571685,
 50345,
 169756,
 38122,
 33992,
 352879,
 188413,
 317062,
 263276,
 213366,
 441441,
 111930,
 41182,
 120917,
 32448,
 210683,
 33451,
 34268,
 36785,
 95220,
 32870,
 294859,
 289250,
 160220,
 94883,
 177689,
 37503,
 35831,
 396912,
 484858,
 27694,
 299879,
 41168,
 47565,
 102394,
 30881,
 575364,
 543020,
 37451,
 314605,
 109121,
 39302,
 74028,
 71768,
 493418,
 661509,
 93343,
 664961,
 3

In [65]:
#print(player_id_list[12])
num_of_ids = len(player_id_list)
print(num_of_ids)

3746


In [66]:
players_leagues_dict = {}

In [67]:
for player_id in  player_id_list:
    if player_id in matchdata.values:
        player_id_data = matchdata[matchdata.values  == player_id]
        players_leagues_dict[player_id]= player_id_data['league_id'].unique().tolist()[0]

In [68]:
print(players_leagues_dict)

{155782: 1729, 27316: 7809, 30895: 1729, 101042: 13274, 40719: 21518, 75489: 1729, 167027: 4769, 128456: 4769, 42664: 10257, 563215: 4769, 41093: 4769, 564712: 4769, 67334: 21518, 39562: 10257, 191784: 21518, 194479: 4769, 352887: 4769, 148827: 21518, 324910: 7809, 24385: 4769, 29904: 21518, 105548: 1729, 240556: 13274, 114503: 13274, 165706: 21518, 213695: 21518, 293235: 10257, 519835: 1729, 89240: 7809, 24159: 1729, 37234: 1729, 209372: 13274, 185336: 7809, 36774: 7809, 357881: 4769, 533212: 4769, 103866: 4769, 47394: 1729, 155738: 10257, 45816: 13274, 77741: 10257, 132844: 7809, 45815: 13274, 449241: 1729, 261599: 13274, 102572: 21518, 482331: 21518, 40501: 21518, 178574: 21518, 23358: 1729, 571685: 21518, 50345: 7809, 169756: 1729, 33992: 21518, 352879: 4769, 188413: 4769, 317062: 4769, 213366: 10257, 441441: 21518, 111930: 1729, 41182: 4769, 120917: 4769, 32448: 1729, 210683: 4769, 33451: 4769, 32870: 1729, 294859: 4769, 289250: 10257, 94883: 7809, 37503: 10257, 396912: 4769, 4848

In [69]:
player_league_df = pd.DataFrame(players_leagues_dict.items(), columns=['player_id', 'league_id'])

In [70]:
print(player_league_df)

      player_id  league_id
0        155782       1729
1         27316       7809
2         30895       1729
3        101042      13274
4         40719      21518
...         ...        ...
2693      35724       4769
2694      42860       7809
2695     107281       7809
2696     282473      10257
2697      26357       4769

[2698 rows x 2 columns]


In [71]:
# Merge player_df and player_league_df

In [72]:
player_df

,player_id,player_name,birthday,height,weight
0,155782,Aaron Cresswell,1989-12-15,170.18,146
12,27316,Aaron Hunt,1986-09-04,182.88,161
21,30895,Aaron Lennon,1987-04-16,165.10,139
31,101042,Aaron Meijers,1987-10-28,175.26,170
39,40719,Aaron Niguez,1989-04-26,170.18,143
...,...,...,...,...,...
33519,35724,Zlatan Ibrahimovic,1981-10-03,195.58,209
33525,42860,Zlatko Junuzovic,1987-09-26,172.72,152
33549,107281,Zoltan Stieber,1988-10-16,175.26,148
33560,282473,Zouhair Feddal,1989-01-01,190.50,172


In [73]:
final_player_df = pd.merge(player_league_df, player_df, on='player_id', how='outer')

In [74]:
final_player_df

,player_id,league_id,player_name,birthday,height,weight
0,155782,1729.0,Aaron Cresswell,1989-12-15,170.18,146
1,27316,7809.0,Aaron Hunt,1986-09-04,182.88,161
2,30895,1729.0,Aaron Lennon,1987-04-16,165.10,139
3,101042,13274.0,Aaron Meijers,1987-10-28,175.26,170
4,40719,21518.0,Aaron Niguez,1989-04-26,170.18,143
...,...,...,...,...,...,...
3741,302079,NaN,Yuya Kubo,1993-12-24,177.80,159
3742,274787,NaN,Yvon Mvogo,1994-06-06,187.96,185
3743,432463,NaN,Zakaria Bakkali,1996-01-26,165.10,132
3744,143750,NaN,Zargo Toure,1989-11-11,182.88,161


In [75]:
final_player_df.isna().sum()

player_id         0
league_id      1048
player_name       0
birthday          0
height            0
weight            0
dtype: int64

In [76]:
final_player_df.dropna(subset=['league_id'],axis=0,inplace=True)

In [77]:
final_player_df

,player_id,league_id,player_name,birthday,height,weight
0,155782,1729.0,Aaron Cresswell,1989-12-15,170.18,146
1,27316,7809.0,Aaron Hunt,1986-09-04,182.88,161
2,30895,1729.0,Aaron Lennon,1987-04-16,165.10,139
3,101042,13274.0,Aaron Meijers,1987-10-28,175.26,170
4,40719,21518.0,Aaron Niguez,1989-04-26,170.18,143
...,...,...,...,...,...,...
2693,35724,4769.0,Zlatan Ibrahimovic,1981-10-03,195.58,209
2694,42860,7809.0,Zlatko Junuzovic,1987-09-26,172.72,152
2695,107281,7809.0,Zoltan Stieber,1988-10-16,175.26,148
2696,282473,10257.0,Zouhair Feddal,1989-01-01,190.50,172


In [78]:
rearranged_players_df = final_player_df[["player_id","player_name","birthday","height","weight","league_id"]]

In [79]:
rearranged_players_df

,player_id,player_name,birthday,height,weight,league_id
0,155782,Aaron Cresswell,1989-12-15,170.18,146,1729.0
1,27316,Aaron Hunt,1986-09-04,182.88,161,7809.0
2,30895,Aaron Lennon,1987-04-16,165.10,139,1729.0
3,101042,Aaron Meijers,1987-10-28,175.26,170,13274.0
4,40719,Aaron Niguez,1989-04-26,170.18,143,21518.0
...,...,...,...,...,...,...
2693,35724,Zlatan Ibrahimovic,1981-10-03,195.58,209,4769.0
2694,42860,Zlatko Junuzovic,1987-09-26,172.72,152,7809.0
2695,107281,Zoltan Stieber,1988-10-16,175.26,148,7809.0
2696,282473,Zouhair Feddal,1989-01-01,190.50,172,10257.0


# The Datasets

In [80]:
rearranged_players_df

,player_id,player_name,birthday,height,weight,league_id
0,155782,Aaron Cresswell,1989-12-15,170.18,146,1729.0
1,27316,Aaron Hunt,1986-09-04,182.88,161,7809.0
2,30895,Aaron Lennon,1987-04-16,165.10,139,1729.0
3,101042,Aaron Meijers,1987-10-28,175.26,170,13274.0
4,40719,Aaron Niguez,1989-04-26,170.18,143,21518.0
...,...,...,...,...,...,...
2693,35724,Zlatan Ibrahimovic,1981-10-03,195.58,209,4769.0
2694,42860,Zlatko Junuzovic,1987-09-26,172.72,152,7809.0
2695,107281,Zoltan Stieber,1988-10-16,175.26,148,7809.0
2696,282473,Zouhair Feddal,1989-01-01,190.50,172,10257.0


In [81]:
rearranged_players_df['league_id'] = rearranged_players_df['league_id'].astype(int)

In [82]:
rearranged_players_df

,player_id,player_name,birthday,height,weight,league_id
0,155782,Aaron Cresswell,1989-12-15,170.18,146,1729
1,27316,Aaron Hunt,1986-09-04,182.88,161,7809
2,30895,Aaron Lennon,1987-04-16,165.10,139,1729
3,101042,Aaron Meijers,1987-10-28,175.26,170,13274
4,40719,Aaron Niguez,1989-04-26,170.18,143,21518
...,...,...,...,...,...,...
2693,35724,Zlatan Ibrahimovic,1981-10-03,195.58,209,4769
2694,42860,Zlatko Junuzovic,1987-09-26,172.72,152,7809
2695,107281,Zoltan Stieber,1988-10-16,175.26,148,7809
2696,282473,Zouhair Feddal,1989-01-01,190.50,172,10257


In [83]:
teams_df1

,team_id,team_short_name,team_long_name,league_id
0,9837,REI,Stade de Reims,4769
1,7794,BAS,SC Bastia,4769
2,4087,ETG,Évian Thonon Gaillard FC,4769
3,9747,GUI,En Avant de Guingamp,4769
4,10249,MON,Montpellier Hérault SC,4769
...,...,...,...,...
1345,8674,GRO,FC Groningen,13274
1348,9908,UTR,FC Utrecht,13274
1352,8277,VIT,Vitesse,13274
1356,8611,TWE,FC Twente,13274


In [84]:
matchstats_df

,id,league_id,season,date,home_team_id,away_team_id,home_team_goal,away_team_goal,goals_info,card_info,corner_info
0,7049,4769,2014/2015,08/08/2014 00:00,9837,9847,2,2,"([[22, 94321, 89476, n], [34, 46653, 159839, n...","[[73, 190972, y], [88, 242958, y]]","[[15, 26852, 3631175], [44, 26852, 3631254], [..."
1,7050,4769,2014/2015,09/08/2014 00:00,7794,8592,3,3,"([[9, 192711, 0, n], [17, 31014, 0, o], [66, 2...","[[19, 42693, y], [64, 280965, y], [82, 40197, ...","[[20, 254308, 3633773], [20, 254308, 3633779],..."
2,7051,4769,2014/2015,09/08/2014 00:00,4087,7819,0,3,"([], [[12, 319300, 289155, n], [32, 210466, 0,...","[[48, 35121, y], [90, 128666, y]]","[[19, 46757, 3633745], [21, 74169, 3633777], [..."
3,7052,4769,2014/2015,09/08/2014 00:00,9747,9853,0,2,"([], [[39, 26374, 0, p], [90, 26374, 0, n]])","[[38, 277412, y], [55, 94323, y], [73, 26051, ...","[[20, 46349, 3633760], [23, 46349, 3633804], [..."
4,7056,4769,2014/2015,09/08/2014 00:00,10249,9827,0,1,"([], [[17, 40707, 352833, n]])","[[7, 139635, y], [33, 154230, y], [63, 355901,...","[[13, 166019, 3633661], [30, 25525, 3633915], ..."
...,...,...,...,...,...,...,...,...,...,...,...
1885,12827,10257,2014/2015,31/05/2015 00:00,8636,8534,4,3,"([[49, 30991, 282770, n], [53, 282770, 239219,...","[[63, 365624, y], [67, 41688, y], [84, 21446, ...","[[6, 21446, 4532573], [22, 21446, 4532792], [3..."
1886,12828,10257,2014/2015,31/05/2015 00:00,7943,10233,3,1,"([[4, 364520, 27685, n], [19, 166595, 364520, ...","[[7, 113318, y], [27, 154239, y], [64, 52243, ...","[[14, 195199, 4532690], [30, 161015, 4532894],..."
1887,12831,10257,2014/2015,31/05/2015 00:00,8686,8540,1,2,"([[85, 30714, 41433, n]], [[35, 270774, 0, p],...","[[56, 103266, y], [68, 108401, y], [77, 41874,...","[[20, 270774, 4532699], [21, 270774, 4532707],..."
1888,12832,10257,2014/2015,31/05/2015 00:00,9882,10167,2,2,"([[54, 362195, 317156, n], [79, 25587, 33639, ...","[[78, 50449, y]]","[[4, 317156, 4532547], [7, 39367, 4532589], [2..."


In [85]:
matchstats_df['card_info'].iloc[0]

[['73', '190972', 'y'], ['88', '242958', 'y']]

In [86]:
league_df

,league_id,league_name
0,4769,France Ligue 1
10,1729,England Premier League
28,7809,Germany 1. Bundesliga
44,21518,Spain LIGA BBVA
84,10257,Italy Serie A
1332,13274,Netherlands Eredivisie


In [87]:
player_attributes_df

,player_id,date,overall_rating,potential,preferred_foot,attacking_work_rate,defensive_work_rate,crossing,finishing,heading_accuracy,...,vision,penalties,marking,standing_tackle,sliding_tackle,gk_diving,gk_handling,gk_kicking,gk_positioning,gk_reflexes
0,155782,2015-12-24,73.0,75.0,left,high,medium,79.0,51.0,57.0,...,65.0,59.0,76.0,75.0,78.0,14.0,7.0,9.0,9.0,12.0
1,27316,2015-09-25,77.0,77.0,left,medium,medium,77.0,72.0,64.0,...,76.0,81.0,28.0,34.0,35.0,15.0,12.0,7.0,16.0,15.0
2,30895,2015-10-16,77.0,77.0,right,high,medium,78.0,66.0,28.0,...,73.0,62.0,30.0,34.0,35.0,14.0,7.0,7.0,16.0,11.0
3,101042,2015-12-03,69.0,69.0,left,medium,medium,63.0,42.0,59.0,...,71.0,56.0,67.0,68.0,65.0,7.0,15.0,7.0,10.0,15.0
4,40719,2015-12-17,71.0,73.0,left,medium,medium,64.0,67.0,54.0,...,62.0,67.0,36.0,31.0,37.0,14.0,13.0,9.0,15.0,10.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3741,35724,2015-09-21,89.0,89.0,right,medium,low,76.0,90.0,76.0,...,83.0,91.0,15.0,41.0,27.0,13.0,15.0,10.0,9.0,12.0
3742,42860,2015-11-26,78.0,78.0,right,high,high,86.0,66.0,58.0,...,78.0,70.0,69.0,70.0,65.0,15.0,9.0,11.0,15.0,8.0
3743,107281,2015-11-06,73.0,74.0,left,medium,medium,69.0,71.0,45.0,...,73.0,65.0,26.0,31.0,32.0,6.0,6.0,9.0,16.0,7.0
3744,282473,2015-11-06,75.0,77.0,left,medium,medium,60.0,40.0,73.0,...,47.0,48.0,78.0,79.0,68.0,13.0,14.0,12.0,10.0,8.0


In [88]:
player_attributes_df["overall_rating"] = player_attributes_df["overall_rating"].astype(int)

In [89]:
player_attributes_df["potential"] = player_attributes_df["potential"].astype(int)

In [90]:
playerid_df = rearranged_players_df[["player_id"]]

In [91]:
playerid_df

,player_id
0,155782
1,27316
2,30895
3,101042
4,40719
...,...
2693,35724
2694,42860
2695,107281
2696,282473


In [92]:
player_attributes_df

,player_id,date,overall_rating,potential,preferred_foot,attacking_work_rate,defensive_work_rate,crossing,finishing,heading_accuracy,...,vision,penalties,marking,standing_tackle,sliding_tackle,gk_diving,gk_handling,gk_kicking,gk_positioning,gk_reflexes
0,155782,2015-12-24,73,75,left,high,medium,79.0,51.0,57.0,...,65.0,59.0,76.0,75.0,78.0,14.0,7.0,9.0,9.0,12.0
1,27316,2015-09-25,77,77,left,medium,medium,77.0,72.0,64.0,...,76.0,81.0,28.0,34.0,35.0,15.0,12.0,7.0,16.0,15.0
2,30895,2015-10-16,77,77,right,high,medium,78.0,66.0,28.0,...,73.0,62.0,30.0,34.0,35.0,14.0,7.0,7.0,16.0,11.0
3,101042,2015-12-03,69,69,left,medium,medium,63.0,42.0,59.0,...,71.0,56.0,67.0,68.0,65.0,7.0,15.0,7.0,10.0,15.0
4,40719,2015-12-17,71,73,left,medium,medium,64.0,67.0,54.0,...,62.0,67.0,36.0,31.0,37.0,14.0,13.0,9.0,15.0,10.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3741,35724,2015-09-21,89,89,right,medium,low,76.0,90.0,76.0,...,83.0,91.0,15.0,41.0,27.0,13.0,15.0,10.0,9.0,12.0
3742,42860,2015-11-26,78,78,right,high,high,86.0,66.0,58.0,...,78.0,70.0,69.0,70.0,65.0,15.0,9.0,11.0,15.0,8.0
3743,107281,2015-11-06,73,74,left,medium,medium,69.0,71.0,45.0,...,73.0,65.0,26.0,31.0,32.0,6.0,6.0,9.0,16.0,7.0
3744,282473,2015-11-06,75,77,left,medium,medium,60.0,40.0,73.0,...,47.0,48.0,78.0,79.0,68.0,13.0,14.0,12.0,10.0,8.0


In [93]:
player_attributes_df1 = pd.merge(player_attributes_df, playerid_df, on="player_id")

In [94]:
player_attributes_df1

,player_id,date,overall_rating,potential,preferred_foot,attacking_work_rate,defensive_work_rate,crossing,finishing,heading_accuracy,...,vision,penalties,marking,standing_tackle,sliding_tackle,gk_diving,gk_handling,gk_kicking,gk_positioning,gk_reflexes
0,155782,2015-12-24,73,75,left,high,medium,79.0,51.0,57.0,...,65.0,59.0,76.0,75.0,78.0,14.0,7.0,9.0,9.0,12.0
1,27316,2015-09-25,77,77,left,medium,medium,77.0,72.0,64.0,...,76.0,81.0,28.0,34.0,35.0,15.0,12.0,7.0,16.0,15.0
2,30895,2015-10-16,77,77,right,high,medium,78.0,66.0,28.0,...,73.0,62.0,30.0,34.0,35.0,14.0,7.0,7.0,16.0,11.0
3,101042,2015-12-03,69,69,left,medium,medium,63.0,42.0,59.0,...,71.0,56.0,67.0,68.0,65.0,7.0,15.0,7.0,10.0,15.0
4,40719,2015-12-17,71,73,left,medium,medium,64.0,67.0,54.0,...,62.0,67.0,36.0,31.0,37.0,14.0,13.0,9.0,15.0,10.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2693,35724,2015-09-21,89,89,right,medium,low,76.0,90.0,76.0,...,83.0,91.0,15.0,41.0,27.0,13.0,15.0,10.0,9.0,12.0
2694,42860,2015-11-26,78,78,right,high,high,86.0,66.0,58.0,...,78.0,70.0,69.0,70.0,65.0,15.0,9.0,11.0,15.0,8.0
2695,107281,2015-11-06,73,74,left,medium,medium,69.0,71.0,45.0,...,73.0,65.0,26.0,31.0,32.0,6.0,6.0,9.0,16.0,7.0
2696,282473,2015-11-06,75,77,left,medium,medium,60.0,40.0,73.0,...,47.0,48.0,78.0,79.0,68.0,13.0,14.0,12.0,10.0,8.0


In [95]:
for col in enumerate(player_attributes_df.columns):
    print(col)

(0, 'player_id')
(1, 'date')
(2, 'overall_rating')
(3, 'potential')
(4, 'preferred_foot')
(5, 'attacking_work_rate')
(6, 'defensive_work_rate')
(7, 'crossing')
(8, 'finishing')
(9, 'heading_accuracy')
(10, 'short_passing')
(11, 'volleys')
(12, 'dribbling')
(13, 'curve')
(14, 'free_kick_accuracy')
(15, 'long_passing')
(16, 'ball_control')
(17, 'acceleration')
(18, 'sprint_speed')
(19, 'agility')
(20, 'reactions')
(21, 'balance')
(22, 'shot_power')
(23, 'jumping')
(24, 'stamina')
(25, 'strength')
(26, 'long_shots')
(27, 'aggression')
(28, 'interceptions')
(29, 'positioning')
(30, 'vision')
(31, 'penalties')
(32, 'marking')
(33, 'standing_tackle')
(34, 'sliding_tackle')
(35, 'gk_diving')
(36, 'gk_handling')
(37, 'gk_kicking')
(38, 'gk_positioning')
(39, 'gk_reflexes')


In [96]:
#Flatten the goals info values 
matchstats_df["goals_info"] = matchstats_df.goals_info.apply(lambda x: sum(x, []))

In [97]:
matchstats_df

,id,league_id,season,date,home_team_id,away_team_id,home_team_goal,away_team_goal,goals_info,card_info,corner_info
0,7049,4769,2014/2015,08/08/2014 00:00,9837,9847,2,2,"[[22, 94321, 89476, n], [34, 46653, 159839, n]...","[[73, 190972, y], [88, 242958, y]]","[[15, 26852, 3631175], [44, 26852, 3631254], [..."
1,7050,4769,2014/2015,09/08/2014 00:00,7794,8592,3,3,"[[9, 192711, 0, n], [17, 31014, 0, o], [66, 21...","[[19, 42693, y], [64, 280965, y], [82, 40197, ...","[[20, 254308, 3633773], [20, 254308, 3633779],..."
2,7051,4769,2014/2015,09/08/2014 00:00,4087,7819,0,3,"[[12, 319300, 289155, n], [32, 210466, 0, n], ...","[[48, 35121, y], [90, 128666, y]]","[[19, 46757, 3633745], [21, 74169, 3633777], [..."
3,7052,4769,2014/2015,09/08/2014 00:00,9747,9853,0,2,"[[39, 26374, 0, p], [90, 26374, 0, n]]","[[38, 277412, y], [55, 94323, y], [73, 26051, ...","[[20, 46349, 3633760], [23, 46349, 3633804], [..."
4,7056,4769,2014/2015,09/08/2014 00:00,10249,9827,0,1,"[[17, 40707, 352833, n]]","[[7, 139635, y], [33, 154230, y], [63, 355901,...","[[13, 166019, 3633661], [30, 25525, 3633915], ..."
...,...,...,...,...,...,...,...,...,...,...,...
1885,12827,10257,2014/2015,31/05/2015 00:00,8636,8534,4,3,"[[49, 30991, 282770, n], [53, 282770, 239219, ...","[[63, 365624, y], [67, 41688, y], [84, 21446, ...","[[6, 21446, 4532573], [22, 21446, 4532792], [3..."
1886,12828,10257,2014/2015,31/05/2015 00:00,7943,10233,3,1,"[[4, 364520, 27685, n], [19, 166595, 364520, n...","[[7, 113318, y], [27, 154239, y], [64, 52243, ...","[[14, 195199, 4532690], [30, 161015, 4532894],..."
1887,12831,10257,2014/2015,31/05/2015 00:00,8686,8540,1,2,"[[85, 30714, 41433, n], [35, 270774, 0, p], [9...","[[56, 103266, y], [68, 108401, y], [77, 41874,...","[[20, 270774, 4532699], [21, 270774, 4532707],..."
1888,12832,10257,2014/2015,31/05/2015 00:00,9882,10167,2,2,"[[54, 362195, 317156, n], [79, 25587, 33639, n...","[[78, 50449, y]]","[[4, 317156, 4532547], [7, 39367, 4532589], [2..."


In [98]:
matchstats_df["corner_info"].iloc[0]

[[15, 26852, 3631175],
 [44, 26852, 3631254],
 [51, 109651, 3631293],
 [52, 89476, 3631297],
 [73, 26852, 3631347],
 [90, 26852, 3631409]]

In [99]:
matchstats_df["goals_info"].iloc[1]

[['9', '192711', '0', 'n'],
 ['17', '31014', '0', 'o'],
 ['66', '213319', '0', 'p'],
 ['73', '192711', '40197', 'n'],
 ['12', '38098', '280965', 'n'],
 ['62', '38098', '0', 'p']]

In [100]:
matchstats_df["card_info"].iloc[1]

[['19', '42693', 'y'],
 ['64', '280965', 'y'],
 ['82', '40197', 'y'],
 ['87', '131991', 'y']]

# Inserting data to postgres DB

In [101]:
def connect_to_db(host_name, dbname, port, username, password):
    try:
        conn = ps.connect(host=host_name, database=dbname, user=username, password=password, port=port)

    except ps.OperationalError as e:
        raise e
    else:
        print('Connected...!')
        return conn

def create_tables(cursor):
    """ create tables in the PostgreSQL database"""
    commands = (
        """
        CREATE TABLE IF NOT EXISTS Country (
                    country_id integer NOT NULL PRIMARY KEY,
                    country_name varchar(255)
            )
            """,
        """
        CREATE TABLE IF NOT EXISTS League (
                    league_id integer NOT NULL PRIMARY KEY,
                    country_id integer,
                    league_name varchar(255),
                    FOREIGN KEY (country_id) REFERENCES Country(country_id)
            )
            """,
        """
        CREATE TABLE IF NOT EXISTS Team (
                    team_id integer NOT NULL PRIMARY KEY,
                    team_short_name varchar(255),
                    team_long_name varchar(255),
                    league_id integer,
                    FOREIGN KEY (league_id) REFERENCES League(league_id)
            )
            """,
        """
        CREATE TABLE IF NOT EXISTS Player (
                    player_id integer NOT NULL PRIMARY KEY,
                    player_name varchar(255),
                    birthday varchar(255),
                    height varchar(255),
                    weight integer,
                    league_id integer,
                    FOREIGN KEY (league_id) REFERENCES League(league_id)
            )
            """,
        """
        CREATE TABLE IF NOT EXISTS Matchstats (
                    id integer NOT NULL PRIMARY KEY,
                    league_id integer,
                    season varchar(255),
                    date varchar(255),
                    home_team_id integer,
                    away_team_id integer,
                    home_team_goal integer,
                    away_team_goal integer,
                    goals_info text[][],
                    card_info text[][],
                    corner_info text[][],
                    FOREIGN KEY (league_id) REFERENCES League(league_id)
            )
            """,
        """
        CREATE TABLE IF NOT EXISTS Player_attributes (
                    attribute_id SERIAL PRIMARY KEY, 
                    player_id integer,
                    season varchar(255),
                    date varchar(255),
                    overall_rating integer,
                    potential integer,
                    preferred_foot varchar(255),
                    attacking_work_rate varchar(255),
                    defensive_work_rate varchar(255),
                    crossing varchar(255),
                    finishing varchar(255),
                    heading_accuracy varchar(255),
                    short_passing varchar(255),
                    volleys varchar(255),
                    dribbling varchar(255),
                    curve varchar(255),
                    free_kick_accuracy varchar(255),
                    long_passing varchar(255),
                    ball_control varchar(255),
                    acceleration varchar(255),
                    sprint_speed varchar(255),
                    agility varchar(255),
                    reactions varchar(255),
                    balance varchar(255),
                    shot_power varchar(255),
                    jumping varchar(255),
                    stamina varchar(255),
                    strength varchar(255),
                    long_shots varchar(255),
                    aggression varchar(255),
                    interceptions varchar(255),
                    positioning varchar(255),
                    vision varchar(255),
                    penalties varchar(255),
                    marking varchar(255),
                    standing_tackle varchar(255),
                    sliding_tackle varchar(255),
                    gk_diving varchar(255),
                    gk_handling varchar(255),
                    gk_kicking varchar(255),
                    gk_positioning varchar(255),
                    gk_reflexes varchar(255),
                    FOREIGN KEY (player_id) REFERENCES Player(player_id)
            )
            """      
        
        )
    try:
        for command in commands:
            cursor.execute(command)
            print("table is created successfully...!")
    except ps.OperationalError as err:
        # pass exception to function
        show_psycopg2_exception(err)
        # set the connection to 'None' in case of error
        conn = None

In [102]:
#Insert into league table
def insert_into_league_table(curr, league_id, league_name):
    insert_into_league = ("""INSERT INTO League (league_id, league_name)
    VALUES(%s,%s);""")
    row_to_insert = (league_id, league_name)
    curr.execute(insert_into_league, row_to_insert)

In [103]:
def append_from_league_df_to_db(curr,df):
    for i, row in df.iterrows():
        insert_into_league_table(curr, row['league_id'], row['league_name'])

In [104]:
#Insert into Team table
def insert_into_team_table(curr, team_id, team_short_name,team_long_name,league_id):
    insert_into_team = ("""INSERT INTO Team (team_id, team_short_name,team_long_name,league_id)
    VALUES(%s,%s,%s,%s);""")
    row_to_insert = (team_id, team_short_name,team_long_name,league_id)
    curr.execute(insert_into_team, row_to_insert)

In [105]:
def append_from_team_df_to_db(curr,df):
    for i, row in df.iterrows():
        insert_into_team_table(curr, row['team_id'], row['team_short_name'],row['team_long_name'],row['league_id'])

In [106]:
#Insert into players table player_id,player_name,birthday,height,weight,league_id
def insert_into_player_table(curr, player_id,player_name,birthday,height,weight,league_id):
    insert_into_player = ("""INSERT INTO Player (player_id,player_name,birthday,height,weight,league_id)
    VALUES(%s,%s,%s,%s,%s,%s);""")
    row_to_insert = (player_id,player_name,birthday,height,weight,league_id)
    curr.execute(insert_into_player, row_to_insert)

In [107]:
def append_from_player_df_to_db(curr,df):
    for i, row in df.iterrows():
        insert_into_player_table(curr, row['player_id'], row['player_name'],row['birthday'],row['height'],row['weight'],row['league_id'])

In [108]:
#Insert into matchstats id,league_id,season,date,home_team_id,away_team_id,home_team_goal,away_team_goal,goals_info,card_info,corner_info
def insert_into_matchstats_table(curr, stat_id,league_id,season,date,home_team_id,away_team_id,home_team_goal,away_team_goal,goals_info,card_info,corner_info):
    insert_into_matchstat = ("""INSERT INTO Matchstats (id,league_id,season,date,home_team_id,away_team_id,home_team_goal,away_team_goal,goals_info,card_info,corner_info)
    VALUES(%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s);""")
    row_to_insert = (stat_id,league_id,season,date,home_team_id,away_team_id,home_team_goal,away_team_goal,goals_info,card_info,corner_info)
    curr.execute(insert_into_matchstat, row_to_insert)

In [109]:
def append_from_matstat_df_to_db(curr,df):
    for i, row in df.iterrows():
            insert_into_matchstats_table(curr, row['id'],row['league_id'],row['season'],row['date'],row['home_team_id'],row['away_team_id'],row['home_team_goal'],row['away_team_goal'],row['goals_info'],row['card_info'],row['corner_info'])

In [110]:
#Insert into player attributes player_id,date,overall_rating,potential,preferred_foot,attacking_work_rate,defensive_work_rate,
#crossing,finishing,heading_accuracy,short_passing,volleys,dribbling,curve,free_kick_accuracy,long_passing,ball_control,acceleration,
#sprint_speed,agility,reactions,balance,shot_power,jumping,stamina,strength,long_shots,aggression,interceptions,positioning,vision,
#penalties,marking,standing_tackle,sliding_tackle,gk_diving,gk_handling,gk_kicking,gk_positioning,gk_reflexes

def insert_into_player_attributes_table(curr, player_id,date,overall_rating,potential,preferred_foot,attacking_work_rate,defensive_work_rate,
                                        crossing,finishing,heading_accuracy,short_passing,volleys,dribbling,curve,free_kick_accuracy,long_passing,ball_control,acceleration,
                                        sprint_speed,agility,reactions,balance,shot_power,jumping,stamina,strength,long_shots,aggression,interceptions,positioning,vision,
                                        penalties,marking,standing_tackle,sliding_tackle,gk_diving,gk_handling,gk_kicking,gk_positioning,gk_reflexes
                                       ):
    insert_into_player_attributes = ("""INSERT INTO Player_attributes (player_id,date,overall_rating,potential,preferred_foot,attacking_work_rate,defensive_work_rate,
                                        crossing,finishing,heading_accuracy,short_passing,volleys,dribbling,curve,free_kick_accuracy,long_passing,ball_control,acceleration,
                                        sprint_speed,agility,reactions,balance,shot_power,jumping,stamina,strength,long_shots,aggression,interceptions,positioning,vision,
                                        penalties,marking,standing_tackle,sliding_tackle,gk_diving,gk_handling,gk_kicking,gk_positioning,gk_reflexes)
    VALUES(%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s);""")
    row_to_insert = (
        player_id,date,overall_rating,potential,preferred_foot,attacking_work_rate,defensive_work_rate,
                                        crossing,finishing,heading_accuracy,short_passing,volleys,dribbling,curve,free_kick_accuracy,long_passing,ball_control,acceleration,
                                        sprint_speed,agility,reactions,balance,shot_power,jumping,stamina,strength,long_shots,aggression,interceptions,positioning,vision,
                                        penalties,marking,standing_tackle,sliding_tackle,gk_diving,gk_handling,gk_kicking,gk_positioning,gk_reflexes
    )
    curr.execute(insert_into_player_attributes, row_to_insert)

In [111]:
def append_from_player_attribute_df_to_db(curr,df):
    for i, row in df.iterrows():
            insert_into_player_attributes_table(curr, row['player_id'],row['date'],row['overall_rating'],row['potential'],row['preferred_foot'],row['attacking_work_rate'],row['defensive_work_rate'],
                                        row['crossing'],row['finishing'],row['heading_accuracy'],row['short_passing'],row['volleys'],row['dribbling'],row['curve'],row['free_kick_accuracy'],row['long_passing'],row['ball_control'],row['acceleration'],
                                        row['sprint_speed'],row['agility'],row['reactions'],row['balance'],row['shot_power'],row['jumping'],row['stamina'],row['strength'],row['long_shots'],row['aggression'],row['interceptions'],row['positioning'],row['vision'],
                                        row['penalties'],row['marking'],row['standing_tackle'],row['sliding_tackle'],row['gk_diving'],row['gk_handling'],row['gk_kicking'],row['gk_positioning'],row['gk_reflexes']
                                               )

In [134]:
#database credentials
host_name = 'ec2-34-230-198-12.compute-1.amazonaws.com'
dbname = 'decsrosnt2dhh0'
port = '5432'
username = 'tenaddwnkksfdp' 
password = '896ad4f701a6055ba52613149b853c90331cf9ff6d4f69ea842c6e452800315c'
conn = None

#host_name = 'localhost'
#dbname = 'european_football_leagues_db'
#port = '5433'
#username = 'postgres' 
#password = 'tony88179'
#conn = None

#establish a connection to db
conn = connect_to_db(host_name, dbname, port, username, password)


Connected...!


In [135]:
curr = conn.cursor()

In [119]:
create_tables(curr)

table is created successfully...!
table is created successfully...!
table is created successfully...!
table is created successfully...!
table is created successfully...!
table is created successfully...!


In [120]:
conn.commit()

In [121]:
#sites_result = curr.fetchall()

In [122]:
curr.close()
conn.close()

In [128]:
league_df

,league_id,league_name
0,4769,France Ligue 1
10,1729,England Premier League
28,7809,Germany 1. Bundesliga
44,21518,Spain LIGA BBVA
84,10257,Italy Serie A
1332,13274,Netherlands Eredivisie


In [129]:
#conn.setAutoCommit(true)

In [136]:
#Insert into leagues table
append_from_league_df_to_db(curr,league_df)


UniqueViolation: duplicate key value violates unique constraint "league_pkey"
DETAIL:  Key (league_id)=(4769) already exists.


In [131]:
conn.commit()

In [138]:
curr.close()
conn.close()

In [133]:
teams_df1

,team_id,team_short_name,team_long_name,league_id
0,9837,REI,Stade de Reims,4769
1,7794,BAS,SC Bastia,4769
2,4087,ETG,Évian Thonon Gaillard FC,4769
3,9747,GUI,En Avant de Guingamp,4769
4,10249,MON,Montpellier Hérault SC,4769
...,...,...,...,...
1345,8674,GRO,FC Groningen,13274
1348,9908,UTR,FC Utrecht,13274
1352,8277,VIT,Vitesse,13274
1356,8611,TWE,FC Twente,13274


In [141]:
#Insert into team table
append_from_team_df_to_db(curr,teams_df1)

In [142]:
conn.commit()

In [143]:
curr.close()
conn.close()

In [114]:
rearranged_players_df

,player_id,player_name,birthday,height,weight,league_id
0,155782,Aaron Cresswell,1989-12-15,170.18,146,1729
1,27316,Aaron Hunt,1986-09-04,182.88,161,7809
2,30895,Aaron Lennon,1987-04-16,165.10,139,1729
3,101042,Aaron Meijers,1987-10-28,175.26,170,13274
4,40719,Aaron Niguez,1989-04-26,170.18,143,21518
...,...,...,...,...,...,...
2693,35724,Zlatan Ibrahimovic,1981-10-03,195.58,209,4769
2694,42860,Zlatko Junuzovic,1987-09-26,172.72,152,7809
2695,107281,Zoltan Stieber,1988-10-16,175.26,148,7809
2696,282473,Zouhair Feddal,1989-01-01,190.50,172,10257


In [115]:
#Insert into players table
append_from_player_df_to_db(curr,rearranged_players_df)

In [116]:
conn.commit()

In [117]:
curr.close()
conn.close()

In [118]:
matchstats_df

,id,league_id,season,date,home_team_id,away_team_id,home_team_goal,away_team_goal,goals_info,card_info,corner_info
0,7049,4769,2014/2015,08/08/2014 00:00,9837,9847,2,2,"[[22, 94321, 89476, n], [34, 46653, 159839, n]...","[[73, 190972, y], [88, 242958, y]]","[[15, 26852, 3631175], [44, 26852, 3631254], [..."
1,7050,4769,2014/2015,09/08/2014 00:00,7794,8592,3,3,"[[9, 192711, 0, n], [17, 31014, 0, o], [66, 21...","[[19, 42693, y], [64, 280965, y], [82, 40197, ...","[[20, 254308, 3633773], [20, 254308, 3633779],..."
2,7051,4769,2014/2015,09/08/2014 00:00,4087,7819,0,3,"[[12, 319300, 289155, n], [32, 210466, 0, n], ...","[[48, 35121, y], [90, 128666, y]]","[[19, 46757, 3633745], [21, 74169, 3633777], [..."
3,7052,4769,2014/2015,09/08/2014 00:00,9747,9853,0,2,"[[39, 26374, 0, p], [90, 26374, 0, n]]","[[38, 277412, y], [55, 94323, y], [73, 26051, ...","[[20, 46349, 3633760], [23, 46349, 3633804], [..."
4,7056,4769,2014/2015,09/08/2014 00:00,10249,9827,0,1,"[[17, 40707, 352833, n]]","[[7, 139635, y], [33, 154230, y], [63, 355901,...","[[13, 166019, 3633661], [30, 25525, 3633915], ..."
...,...,...,...,...,...,...,...,...,...,...,...
1885,12827,10257,2014/2015,31/05/2015 00:00,8636,8534,4,3,"[[49, 30991, 282770, n], [53, 282770, 239219, ...","[[63, 365624, y], [67, 41688, y], [84, 21446, ...","[[6, 21446, 4532573], [22, 21446, 4532792], [3..."
1886,12828,10257,2014/2015,31/05/2015 00:00,7943,10233,3,1,"[[4, 364520, 27685, n], [19, 166595, 364520, n...","[[7, 113318, y], [27, 154239, y], [64, 52243, ...","[[14, 195199, 4532690], [30, 161015, 4532894],..."
1887,12831,10257,2014/2015,31/05/2015 00:00,8686,8540,1,2,"[[85, 30714, 41433, n], [35, 270774, 0, p], [9...","[[56, 103266, y], [68, 108401, y], [77, 41874,...","[[20, 270774, 4532699], [21, 270774, 4532707],..."
1888,12832,10257,2014/2015,31/05/2015 00:00,9882,10167,2,2,"[[54, 362195, 317156, n], [79, 25587, 33639, n...","[[78, 50449, y]]","[[4, 317156, 4532547], [7, 39367, 4532589], [2..."


In [121]:
append_from_matstat_df_to_db(curr,matchstats_df)

In [122]:
conn.commit()

In [123]:
curr.close()
conn.close()

In [126]:
player_attributes_df1

,player_id,date,overall_rating,potential,preferred_foot,attacking_work_rate,defensive_work_rate,crossing,finishing,heading_accuracy,...,vision,penalties,marking,standing_tackle,sliding_tackle,gk_diving,gk_handling,gk_kicking,gk_positioning,gk_reflexes
0,155782,2015-12-24,73,75,left,high,medium,79.0,51.0,57.0,...,65.0,59.0,76.0,75.0,78.0,14.0,7.0,9.0,9.0,12.0
1,27316,2015-09-25,77,77,left,medium,medium,77.0,72.0,64.0,...,76.0,81.0,28.0,34.0,35.0,15.0,12.0,7.0,16.0,15.0
2,30895,2015-10-16,77,77,right,high,medium,78.0,66.0,28.0,...,73.0,62.0,30.0,34.0,35.0,14.0,7.0,7.0,16.0,11.0
3,101042,2015-12-03,69,69,left,medium,medium,63.0,42.0,59.0,...,71.0,56.0,67.0,68.0,65.0,7.0,15.0,7.0,10.0,15.0
4,40719,2015-12-17,71,73,left,medium,medium,64.0,67.0,54.0,...,62.0,67.0,36.0,31.0,37.0,14.0,13.0,9.0,15.0,10.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2693,35724,2015-09-21,89,89,right,medium,low,76.0,90.0,76.0,...,83.0,91.0,15.0,41.0,27.0,13.0,15.0,10.0,9.0,12.0
2694,42860,2015-11-26,78,78,right,high,high,86.0,66.0,58.0,...,78.0,70.0,69.0,70.0,65.0,15.0,9.0,11.0,15.0,8.0
2695,107281,2015-11-06,73,74,left,medium,medium,69.0,71.0,45.0,...,73.0,65.0,26.0,31.0,32.0,6.0,6.0,9.0,16.0,7.0
2696,282473,2015-11-06,75,77,left,medium,medium,60.0,40.0,73.0,...,47.0,48.0,78.0,79.0,68.0,13.0,14.0,12.0,10.0,8.0


In [136]:
append_from_player_attribute_df_to_db(curr,player_attributes_df1)

In [137]:
conn.commit()

In [138]:
curr.close()
conn.close()